In [ ]:
%pylab inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import keras.models as km
import keras.layers as kl
from sklearn import metrics, model_selection

In [ ]:
ratings = pd.read_csv('data_train.csv', dtype={'Prediction': np.int})
pos = ratings.Id.str.extract('r([0-9]+)_c([0-9]+)', expand=True)
ratings['User'] = pos[0]
ratings['Item'] = pos[1]
ratings.head()

In [ ]:
n_i = ratings.Item.unique().shape[0]
n_i

In [ ]:
n_u = ratings.User.unique().shape[0]
n_u

In [ ]:
y = np.zeros([ratings.shape[0], 5])
y[np.arange(ratings.shape[0]), ratings.Prediction - 1] = 1
y.shape

In [ ]:
input_i = kl.Input(shape=[1])
i = kl.Embedding(n_i + 1, 64)(input_i)
i = kl.Flatten()(i)
i = kl.Dropout(0.5)(i)

input_u = kl.Input(shape=[1])
u = kl.Embedding(n_u + 1, 64)(input_u)
u = kl.Flatten()(u)
u = kl.Dropout(0.5)(u)

nn = kl.merge([i, u], mode='concat')
nn = kl.Dense(128, activation='relu')(nn)
nn = kl.Dropout(0.5)(nn)
nn = kl.normalization.BatchNormalization()(nn)
nn = kl.Dense(128, activation='relu')(nn)
#nn = kl.Dropout(0.5)(nn)
#nn = kl.normalization.BatchNormalization()(nn)
#nn = kl.Dense(128, activation='relu')(nn)

output = kl.Dense(5, activation='softmax')(nn)

model = km.Model([input_i, input_u], output)
model.compile('adam', 'categorical_crossentropy')

In [ ]:
train_u, test_u, train_i, test_i, train_y, test_y = model_selection.train_test_split(ratings.User, ratings.Item, y, test_size=0.5)
train_u.shape

In [ ]:
epochs = model.fit(
    [train_i[:], train_u[:]], train_y[:], 
    validation_split=0.1,
    nb_epoch=3
)

In [ ]:
mse = metrics.mean_squared_error(
    np.argmax(test_y, 1) + 1, 
    np.argmax(model.predict([test_i, test_u]), 1) + 1
)
np.sqrt(mse)

In [ ]:
sub = pd.read_csv('sampleSubmission.csv')
sub.head()

In [ ]:
pos = sub.Id.str.extract('r([0-9]+)_c([0-9]+)', expand=True)
sub['User'] = pos[0]
sub['Item'] = pos[1]
sub.head()

In [ ]:
sub['Prediction'] = np.argmax(model.predict([sub.Item, sub.User]), axis=1) + 1
sub.head()

In [ ]:
sub.to_csv('submission.csv', columns=['Id', 'Prediction'], index=False)